In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
from datetime import datetime
import matplotlib.pylab as pylab
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei']


In [2]:
#首先导入用户基本信息，作为每个月的样本，构造训练集和测试集
#用户基本信息表--jdata_user_basic_info
jdata_user_basic_info_data = pd.read_csv('jdata_user_basic_info.csv')
print(jdata_user_basic_info_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98924 entries, 0 to 98923
Data columns (total 4 columns):
user_id       98924 non-null int64
age           98924 non-null int64
sex           98924 non-null int64
user_lv_cd    98924 non-null int64
dtypes: int64(4)
memory usage: 3.0 MB
None


In [3]:
#用户订单表--jdata_user_order
jdata_user_order_data = pd.read_csv('jdata_user_order.csv')
print(jdata_user_order_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792723 entries, 0 to 792722
Data columns (total 6 columns):
user_id      792723 non-null int64
sku_id       792723 non-null int64
o_id         792723 non-null int64
o_date       792723 non-null object
o_area       792723 non-null int64
o_sku_num    792723 non-null int64
dtypes: int64(5), object(1)
memory usage: 36.3+ MB
None


In [5]:
#导入剩余的3张表
#用户行为表--jdata_user_action
jdata_user_action_data = pd.read_csv('jdata_user_action.csv')
print(jdata_user_action_data.info())

#SKU基本信息表--jdata_sku_basic_info
jdata_sku_basic_info_data = pd.read_csv('jdata_sku_basic_info.csv')
print(jdata_sku_basic_info_data.info())

#评论分数数据表--jdata_user_comment_score
jdata_user_comment_score_data = pd.read_csv('jdata_user_comment_score.csv')
print(jdata_user_comment_score_data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6944141 entries, 0 to 6944140
Data columns (total 5 columns):
user_id    int64
sku_id     int64
a_date     object
a_num      int64
a_type     int64
dtypes: int64(4), object(1)
memory usage: 264.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99412 entries, 0 to 99411
Data columns (total 6 columns):
sku_id    99412 non-null int64
price     99412 non-null float64
cate      99412 non-null int64
para_1    99412 non-null float64
para_2    99412 non-null int64
para_3    99412 non-null int64
dtypes: float64(2), int64(4)
memory usage: 4.6 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224284 entries, 0 to 224283
Data columns (total 4 columns):
user_id              224284 non-null int64
comment_create_tm    224284 non-null object
o_id                 224284 non-null int64
score_level          224284 non-null int64
dtypes: int64(3), object(1)
memory usage: 6.8+ MB
None


In [6]:
#定义时间处理函数
def dealDateColumns(df, columnName):
    df[columnName] = pd.to_datetime(df[columnName])
    df['year'] = df[columnName].map(lambda x: x.year)
    df['month'] = df[columnName].map(lambda x: x.month)
    df['day'] = df[columnName].map(lambda x: x.day)
    return df

jdata_user_order_data = dealDateColumns(jdata_user_order_data, 'o_date')
jdata_user_action_data = dealDateColumns(jdata_user_action_data, 'a_date')
jdata_user_comment_score_data = dealDateColumns(jdata_user_comment_score_data, 'comment_create_tm')


In [7]:
#首先过滤出目标类目订单记录用于训练集打标签
jdata_user_order_data = pd.merge(jdata_user_order_data, jdata_sku_basic_info_data[['sku_id', 'cate']], on=['sku_id'], how='left')
jdata_user_action_data = pd.merge(jdata_user_action_data, jdata_sku_basic_info_data[['sku_id', 'cate']], on=['sku_id'], how='left')
jdata_user_order_data_target = jdata_user_order_data[(jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)]
print(len(jdata_user_order_data))
print(len(jdata_user_order_data_target))


# #首先尝试对每个月构造训练集，并为每条样本打上标签
# def getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, month, year, end_time):
#     user_set = set(jdata_user_order_data_target['user_id'][(jdata_user_order_data.o_date < end_time)])
#     jdata_train_df_1 = pd.DataFrame(columns=['user_id', 'age', 'sex', 'user_lv_cd'])
#     for user_id, age, sex, user_lv_cd in jdata_user_basic_info_data[['user_id', 'age', 'sex', 'user_lv_cd']].values:
#         if user_id in user_set:
#             insertRow = pd.DataFrame([[user_id, age, sex, user_lv_cd]], columns=['user_id', 'age', 'sex', 'user_lv_cd'])
#             jdata_train_df_1 = jdata_train_df_1.append(insertRow)
#     print(len(jdata_train_df_1))
#     jdata_train_df_1['month'] = month
#     jdata_train_df_1['year'] = year
#     jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_target[['user_id', 'year', 'month', 'day', 'o_date']], on=['user_id', 'year', 'month'], how='left')
#     jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=True)
#     jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
#     jdata_train_df_1['o_date'] = jdata_train_df_1['o_date'].fillna(pd.to_datetime('2000-01-01'))
#     jdata_train_df_1['is_order'] = jdata_train_df_1['day'].map(lambda x: 0 if math.isnan(x) else 1)
#     jdata_train_df_1['order_day'] = jdata_train_df_1['day'].map(lambda x: -1 if math.isnan(x) else x)
#     del jdata_train_df_1['day']
#     del jdata_train_df_1['o_date']
#     print(year, month, len(jdata_train_df_1[jdata_train_df_1.is_order == 1]))
# #     jdata_train_df = pd.concat([jdata_train_df, jdata_train_df_1])
#     return jdata_train_df_1

# jdata_train_df_201704 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 4, 2017, datetime.datetime(2017, 4, 1))
# jdata_train_df_201703 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 3, 2017, datetime.datetime(2017, 3, 1))
# jdata_train_df_201702 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 2, 2017, datetime.datetime(2017, 2, 1))
# jdata_train_df_201701 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 1, 2017, datetime.datetime(2017, 1, 1))
# jdata_train_df_201612 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 12, 2016, datetime.datetime(2016, 12, 1))
# jdata_train_df_201611 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 11, 2016, datetime.datetime(2016, 11, 1))
# print(jdata_train_df_201704.head(10))

def getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, month, year):
    jdata_train_df_1 = jdata_user_basic_info_data.copy()
    jdata_train_df_1['month'] = month
    jdata_train_df_1['year'] = year
    jdata_train_df_1 = pd.merge(jdata_train_df_1, jdata_user_order_data_target[['user_id', 'year', 'month', 'day', 'o_date']], on=['user_id', 'year', 'month'], how='left')
    jdata_train_df_1 = jdata_train_df_1.sort_index(by=['user_id', 'o_date'], ascending=True)
    jdata_train_df_1 = jdata_train_df_1.drop_duplicates(['user_id'])
    jdata_train_df_1['o_date'] = jdata_train_df_1['o_date'].fillna(pd.to_datetime('2000-01-01'))
    jdata_train_df_1['is_order'] = jdata_train_df_1['day'].map(lambda x: 0 if math.isnan(x) else 1)
    jdata_train_df_1['order_day'] = jdata_train_df_1['day'].map(lambda x: -1 if math.isnan(x) else x)
    del jdata_train_df_1['day']
    del jdata_train_df_1['o_date']
    print(year, month, len(jdata_train_df_1[jdata_train_df_1.is_order == 1]))
#     jdata_train_df = pd.concat([jdata_train_df, jdata_train_df_1])
    return jdata_train_df_1

jdata_train_df_201704 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 4, 2017)
jdata_train_df_201703 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 3, 2017)
jdata_train_df_201702 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 2, 2017)
jdata_train_df_201701 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 1, 2017)
jdata_train_df_201612 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 12, 2016)
jdata_train_df_201611 = getTrainDfMonthly(jdata_user_order_data_target, jdata_user_basic_info_data, 11, 2016)
print(jdata_train_df_201704.head(10))



792723
550378


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


2017 4 53553
2017 3 52574
2017 2 47684
2017 1 31030
2016 12 28760
2016 11 31341
    user_id  age  sex  user_lv_cd  month  year  is_order  order_day
0         1    3    2           3      4  2017         1       14.0
2         2    3    2           3      4  2017         1       24.0
3         3    3    0           5      4  2017         1       26.0
4         4    3    2           3      4  2017         1       22.0
5         5    4    2           2      4  2017         1       11.0
6         6   -1    2           2      4  2017         1       12.0
7         7    3    2           4      4  2017         1        7.0
8         8    2    2           2      4  2017         0       -1.0
9         9    3    2           3      4  2017         0       -1.0
10       10    4    0           4      4  2017         0       -1.0


In [8]:
#构造测试集
jdata_test_df = jdata_user_basic_info_data.copy()
jdata_test_df['year'] = 2017
jdata_test_df['month'] = 5
print(len(jdata_test_df))
print(jdata_test_df.head(10))


98924
   user_id  age  sex  user_lv_cd  year  month
0        1    3    2           3  2017      5
1        2    3    2           3  2017      5
2        3    3    0           5  2017      5
3        4    3    2           3  2017      5
4        5    4    2           2  2017      5
5        6   -1    2           2  2017      5
6        7    3    2           4  2017      5
7        8    2    2           2  2017      5
8        9    3    2           3  2017      5
9       10    4    0           4  2017      5


In [9]:
# 为训练集和测试集定义滑窗区间
FeatureMonthList_201704 = [(datetime.datetime(2017, 3, 1), datetime.datetime(2017, 4, 1), 1), 
                          (datetime.datetime(2017, 3, 15), datetime.datetime(2017, 4, 1), 0.5),
                          (datetime.datetime(2017, 2, 1), datetime.datetime(2017, 3, 1), 'one'),
                          (datetime.datetime(2017, 1, 1), datetime.datetime(2017, 4, 1), 3),
                          (datetime.datetime(2016, 10, 1), datetime.datetime(2017, 4, 1), 6)]

FeatureMonthList_201703 = [(datetime.datetime(2017, 2, 1), datetime.datetime(2017, 3, 1), 1),
                          (datetime.datetime(2017, 2, 15), datetime.datetime(2017, 3, 1), 0.5),
                          (datetime.datetime(2017, 1, 1), datetime.datetime(2017, 2, 1), 'one'),
                          (datetime.datetime(2016, 12, 1), datetime.datetime(2017, 3, 1), 3),
                          (datetime.datetime(2016, 9, 1), datetime.datetime(2017, 3, 1), 6)]

FeatureMonthList_201702 = [(datetime.datetime(2017, 1, 1), datetime.datetime(2017, 2, 1), 1), 
                          (datetime.datetime(2017, 1, 15), datetime.datetime(2017, 2, 1), 0.5),
                          (datetime.datetime(2016, 12, 1), datetime.datetime(2017, 1, 1), 'one'),
                          (datetime.datetime(2016, 11, 1), datetime.datetime(2017, 2, 1), 3),
                          (datetime.datetime(2016, 8, 1), datetime.datetime(2017, 2, 1), 6)]

FeatureMonthList_201701 = [(datetime.datetime(2016, 12, 1), datetime.datetime(2017, 1, 1), 1), 
                          (datetime.datetime(2016, 12, 15), datetime.datetime(2017, 1, 1), 0.5),
                          (datetime.datetime(2016, 11, 1), datetime.datetime(2016, 12, 1), 'one'),
                          (datetime.datetime(2016, 10, 1), datetime.datetime(2017, 1, 1), 3),
                          (datetime.datetime(2016, 7, 1), datetime.datetime(2017, 1, 1), 6)]

FeatureMonthList_201612 = [(datetime.datetime(2016, 11, 1), datetime.datetime(2016, 12, 1), 1), 
                           (datetime.datetime(2016, 11, 15), datetime.datetime(2016, 12, 1), 0.5),
                          (datetime.datetime(2016, 10, 1), datetime.datetime(2016, 11, 1), 'one'),
                          (datetime.datetime(2016, 9, 1), datetime.datetime(2016, 12, 1), 3),
                          (datetime.datetime(2016, 6, 1), datetime.datetime(2016, 12, 1), 6)]

FeatureMonthList_201611 = [(datetime.datetime(2016, 10, 1), datetime.datetime(2016, 11, 1), 1), 
                          (datetime.datetime(2016, 10, 15), datetime.datetime(2016, 11, 1), 0.5),
                          (datetime.datetime(2016, 9, 1), datetime.datetime(2016, 10, 1), 'one'),
                          (datetime.datetime(2016, 8, 1), datetime.datetime(2016, 11, 1), 3),
                          (datetime.datetime(2016, 5, 1), datetime.datetime(2016, 11, 1), 6)]

FeatureMonthList_201705 = [(datetime.datetime(2017, 4, 1), datetime.datetime(2017, 5, 1), 1), 
                          (datetime.datetime(2017, 4, 15), datetime.datetime(2017, 5, 1), 0.5),
                          (datetime.datetime(2017, 3, 1), datetime.datetime(2017, 4, 1), 'one'),
                          (datetime.datetime(2017, 2, 1), datetime.datetime(2017, 5, 1), 3),
                          (datetime.datetime(2016, 11, 1), datetime.datetime(2017, 5, 1), 6)]


In [10]:
#　添加一个冷启动特征，用户是否是上个月第一次购买特征
def getUserIsLastMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_df_copy = jdata_df.copy()
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
        jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_one_pivot_table = pd.pivot_table(jdata_user_order_data_one, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_one_pivot_table.rename(columns={'o_id':'one_count'}, inplace=True)
    jdata_user_order_data_one_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pre_pivot_table = pd.pivot_table(jdata_user_order_data_pre, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_pre_pivot_table.rename(columns={'o_id':'pre_count'}, inplace=True)
    jdata_user_order_data_pre_pivot_table.reset_index(inplace=True)
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_one_pivot_table, on=['user_id'], how='left')
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_pre_pivot_table, on=['user_id'], how='left')
    jdata_df_copy['one_count'] = jdata_df_copy['one_count'].fillna(-1)
    jdata_df_copy['pre_count'] = jdata_df_copy['pre_count'].fillna(-1)
    jdata_df_copy['user_cate' + str(cate) + '_isLastMonthFirstBuy'] = 0
    jdata_df_copy['user_cate' + str(cate) + '_isLastMonthFirstBuy'][(jdata_df_copy.one_count != -1) & (jdata_df_copy.pre_count == -1)] = 1
    jdata_df = pd.merge(jdata_df, jdata_df_copy[['user_id', 'user_cate' + str(cate) + '_isLastMonthFirstBuy']], on=['user_id'], how='left')
    return jdata_df

def getUserIsLastMonthFirstBuy(jdata_df, jdata_user_order_data, start_time, end_time):
    jdata_df = getUserIsLastMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -1)
    jdata_df = getUserIsLastMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -2)
    jdata_df = getUserIsLastMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 101)
    jdata_df = getUserIsLastMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 30)
    return jdata_df

jdata_train_df_201704 = getUserIsLastMonthFirstBuy(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 3, 1), datetime.datetime(2017, 4, 1))
jdata_train_df_201703 = getUserIsLastMonthFirstBuy(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 3, 1))
jdata_train_df_201702 = getUserIsLastMonthFirstBuy(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 2, 1))
jdata_train_df_201701 = getUserIsLastMonthFirstBuy(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 1, 1))
jdata_train_df_201612 = getUserIsLastMonthFirstBuy(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2016, 12, 1))
jdata_train_df_201611 = getUserIsLastMonthFirstBuy(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2016, 11, 1))
jdata_test_df = getUserIsLastMonthFirstBuy(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 4, 1), datetime.datetime(2017, 5, 1))


# set1 = set(jdata_user_order_data['user_id'][(jdata_user_order_data.o_date < datetime.datetime(2017, 4, 1)) & (jdata_user_order_data.o_date >= datetime.datetime(2017, 3, 1)) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))])
# set2 = set(jdata_user_order_data['user_id'][(jdata_user_order_data.o_date < datetime.datetime(2017, 3, 1)) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))])
# print(len(set1))
# print(len(set2))
# print(len(set1 & set2))


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
#　添加一个冷启动特征，用户是否是上2个月第一次购买特征
def getUserIsLastTwoMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_df_copy = jdata_df.copy()
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
        jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_one_pivot_table = pd.pivot_table(jdata_user_order_data_one, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_one_pivot_table.rename(columns={'o_id':'one_count'}, inplace=True)
    jdata_user_order_data_one_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pre_pivot_table = pd.pivot_table(jdata_user_order_data_pre, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_pre_pivot_table.rename(columns={'o_id':'pre_count'}, inplace=True)
    jdata_user_order_data_pre_pivot_table.reset_index(inplace=True)
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_one_pivot_table, on=['user_id'], how='left')
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_pre_pivot_table, on=['user_id'], how='left')
    jdata_df_copy['one_count'] = jdata_df_copy['one_count'].fillna(-1)
    jdata_df_copy['pre_count'] = jdata_df_copy['pre_count'].fillna(-1)
    jdata_df_copy['user_cate' + str(cate) + '_isLastTwoMonthFirstBuy'] = 0
    jdata_df_copy['user_cate' + str(cate) + '_isLastTwoMonthFirstBuy'][(jdata_df_copy.one_count != -1) & (jdata_df_copy.pre_count == -1)] = 1
    jdata_df = pd.merge(jdata_df, jdata_df_copy[['user_id', 'user_cate' + str(cate) + '_isLastTwoMonthFirstBuy']], on=['user_id'], how='left')
    return jdata_df

def getUserIsLastTwoMonthFirstBuy(jdata_df, jdata_user_order_data, start_time, end_time):
    jdata_df = getUserIsLastTwoMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -1)
    jdata_df = getUserIsLastTwoMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -2)
    jdata_df = getUserIsLastTwoMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 101)
    jdata_df = getUserIsLastTwoMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 30)
    return jdata_df

jdata_train_df_201704 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 3, 1))
jdata_train_df_201703 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 2, 1))
jdata_train_df_201702 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 1, 1))
jdata_train_df_201701 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2016, 12, 1))
jdata_train_df_201612 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2016, 11, 1))
jdata_train_df_201611 = getUserIsLastTwoMonthFirstBuy(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 10, 1))
jdata_test_df = getUserIsLastTwoMonthFirstBuy(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 3, 1), datetime.datetime(2017, 4, 1))



/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
#　添加一个冷启动特征，用户是否是上3个月第一次购买特征
def getUserIsLastThreeMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate):
    jdata_df_copy = jdata_df.copy()
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
            jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_one = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
        jdata_user_order_data_pre = jdata_user_order_data[(jdata_user_order_data.o_date < start_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_one_pivot_table = pd.pivot_table(jdata_user_order_data_one, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_one_pivot_table.rename(columns={'o_id':'one_count'}, inplace=True)
    jdata_user_order_data_one_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_pre_pivot_table = pd.pivot_table(jdata_user_order_data_pre, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_pre_pivot_table.rename(columns={'o_id':'pre_count'}, inplace=True)
    jdata_user_order_data_pre_pivot_table.reset_index(inplace=True)
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_one_pivot_table, on=['user_id'], how='left')
    jdata_df_copy = pd.merge(jdata_df_copy, jdata_user_order_data_pre_pivot_table, on=['user_id'], how='left')
    jdata_df_copy['one_count'] = jdata_df_copy['one_count'].fillna(-1)
    jdata_df_copy['pre_count'] = jdata_df_copy['pre_count'].fillna(-1)
    jdata_df_copy['user_cate' + str(cate) + '_isLastThreeMonthFirstBuy'] = 0
    jdata_df_copy['user_cate' + str(cate) + '_isLastThreeMonthFirstBuy'][(jdata_df_copy.one_count != -1) & (jdata_df_copy.pre_count == -1)] = 1
    jdata_df = pd.merge(jdata_df, jdata_df_copy[['user_id', 'user_cate' + str(cate) + '_isLastThreeMonthFirstBuy']], on=['user_id'], how='left')
    return jdata_df

def getUserIsLastThreeMonthFirstBuy(jdata_df, jdata_user_order_data, start_time, end_time):
    jdata_df = getUserIsLastThreeMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -1)
    jdata_df = getUserIsLastThreeMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, -2)
    jdata_df = getUserIsLastThreeMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 101)
    jdata_df = getUserIsLastThreeMonthFirstBuyMonthly(jdata_df, jdata_user_order_data, start_time, end_time, 30)
    return jdata_df

jdata_train_df_201704 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 1, 1), datetime.datetime(2017, 2, 1))
jdata_train_df_201703 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2016, 12, 1), datetime.datetime(2017, 1, 1))
jdata_train_df_201702 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2016, 11, 1), datetime.datetime(2016, 12, 1))
jdata_train_df_201701 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2016, 10, 1), datetime.datetime(2016, 11, 1))
jdata_train_df_201612 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 9, 1), datetime.datetime(2016, 10, 1))
jdata_train_df_201611 = getUserIsLastThreeMonthFirstBuy(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 8, 1), datetime.datetime(2016, 9, 1))
jdata_test_df = getUserIsLastThreeMonthFirstBuy(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 2, 1), datetime.datetime(2017, 3, 1))



/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
# 统计某用户在滑窗区间内下单某个类目商品的次数，逐个月进行处理
def getUserOrderNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_order_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_order_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_order_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_order_future.drop_duplicates(['user_id', 'o_id'], inplace=True)
    jdata_user_order_data_order_future_pivot_table = pd.pivot_table(jdata_user_order_data_order_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_order_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_order_future_pivot_table.rename(columns={'o_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_order_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserOrderNumber(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserOrderNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_orderNumber')
        jdata_df = getUserOrderNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_orderNumber')
        jdata_df = getUserOrderNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_orderNumber')
        jdata_df = getUserOrderNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_orderNumber')
    return jdata_df

jdata_train_df_201704 = getUserOrderNumber(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserOrderNumber(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserOrderNumber(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserOrderNumber(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserOrderNumber(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserOrderNumber(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserOrderNumber(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)
                                                                                                           

/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# 统计某用户在滑窗区间内购买某个类目商品的次数，逐个月进行处理
def getUserBuyNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserBuyNumber(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyNumber')
        jdata_df = getUserBuyNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyNumber')
        jdata_df = getUserBuyNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyNumber')
        jdata_df = getUserBuyNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyNumber')
    return jdata_df

jdata_train_df_201704 = getUserBuyNumber(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyNumber(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyNumber(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyNumber(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyNumber(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyNumber(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyNumber(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内购买某个类目商品的天数，逐个月进行处理
def getUserBuyDayNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future.drop_duplicates(['user_id', 'o_date'], inplace=True)
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserBuyDayNumber(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyDayNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyDayNumber')
        jdata_df = getUserBuyDayNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyDayNumber')
        jdata_df = getUserBuyDayNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyDayNumber')
        jdata_df = getUserBuyDayNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyDayNumber')
    return jdata_df

jdata_train_df_201704 = getUserBuyDayNumber(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyDayNumber(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyDayNumber(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyDayNumber(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyDayNumber(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyDayNumber(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyDayNumber(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


/home/lab-zhao.yinhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
# 统计某用户在滑窗区间内购买某个类目商品的件数，逐个月进行处理
def getUserBuyCountMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserBuyCount(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyCountMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyCount')
        jdata_df = getUserBuyCountMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyCount')
        jdata_df = getUserBuyCountMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyCount')
        jdata_df = getUserBuyCountMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyCount')
    return jdata_df

jdata_train_df_201704 = getUserBuyCount(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyCount(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyCount(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyCount(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyCount(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyCount(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyCount(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内购买某个类目商品在当月第几天的一些统计特征(min,max,mean)，逐个月进行处理
def getUserBuyDayMinMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['day'], aggfunc=min)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'day':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    return jdata_train_df_dealMonth

def getUserBuyDayMin(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyDayMinMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyDayMin')
        jdata_df = getUserBuyDayMinMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyDayMin')
        jdata_df = getUserBuyDayMinMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyDayMin')
        jdata_df = getUserBuyDayMinMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyDayMin')
    return jdata_df

jdata_train_df_201704 = getUserBuyDayMin(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyDayMin(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyDayMin(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyDayMin(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyDayMin(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyDayMin(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyDayMin(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
def getUserBuyDayMaxMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['day'], aggfunc=max)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'day':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    return jdata_train_df_dealMonth

def getUserBuyDayMax(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyDayMaxMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyDayMax')
        jdata_df = getUserBuyDayMaxMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyDayMax')
        jdata_df = getUserBuyDayMaxMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyDayMax')
        jdata_df = getUserBuyDayMaxMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyDayMax')
    return jdata_df

jdata_train_df_201704 = getUserBuyDayMax(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyDayMax(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyDayMax(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyDayMax(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyDayMax(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyDayMax(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyDayMax(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
def getUserBuyDayMeanMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['day'], aggfunc=mean)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'day':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    return jdata_train_df_dealMonth

def getUserBuyDayMean(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyDayMeanMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyDayMean')
        jdata_df = getUserBuyDayMeanMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyDayMean')
        jdata_df = getUserBuyDayMeanMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyDayMean')
        jdata_df = getUserBuyDayMeanMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyDayMean')
    return jdata_df

jdata_train_df_201704 = getUserBuyDayMean(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyDayMean(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyDayMean(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyDayMean(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyDayMean(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyDayMean(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyDayMean(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内购买某个类目商品的月份数，逐个月进行处理
def getUserBuyMonthNumberMonthly(jdata_df, jdata_user_order_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.o_date >= start_time) & (jdata_user_order_data.o_date < end_time) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_buy_future.drop_duplicates(['user_id', 'month'], inplace=True)
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['month'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'month':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserBuyMonthNumber(jdata_df, jdata_user_order_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBuyMonthNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_buyMonthNumber')
        jdata_df = getUserBuyMonthNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_buyMonthNumber')
        jdata_df = getUserBuyMonthNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_buyMonthNumber')
        jdata_df = getUserBuyMonthNumberMonthly(jdata_df, jdata_user_order_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_buyMonthNumber')
    return jdata_df

jdata_train_df_201704 = getUserBuyMonthNumber(jdata_train_df_201704, jdata_user_order_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBuyMonthNumber(jdata_train_df_201703, jdata_user_order_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBuyMonthNumber(jdata_train_df_201702, jdata_user_order_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBuyMonthNumber(jdata_train_df_201701, jdata_user_order_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBuyMonthNumber(jdata_train_df_201612, jdata_user_order_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBuyMonthNumber(jdata_train_df_201611, jdata_user_order_data, FeatureMonthList_201611)
jdata_test_df = getUserBuyMonthNumber(jdata_test_df, jdata_user_order_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的操作个数，逐个月进行处理
def getUserSkuActionNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'sku_id'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserSkuActionNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserSkuActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_skuActionNumber')
        jdata_df = getUserSkuActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_skuActionNumber')
        jdata_df = getUserSkuActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_skuActionNumber')
        jdata_df = getUserSkuActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_skuActionNumber')
    return jdata_df

jdata_train_df_201704 = getUserSkuActionNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserSkuActionNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserSkuActionNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserSkuActionNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserSkuActionNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserSkuActionNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserSkuActionNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的操作天数，逐个月进行处理
def getUserSkuActionDayNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'a_date'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_date'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_date':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserSkuActionDayNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserSkuActionDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_skuActionDayNumber')
        jdata_df = getUserSkuActionDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_skuActionDayNumber')
        jdata_df = getUserSkuActionDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_skuActionDayNumber')
        jdata_df = getUserSkuActionDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_skuActionDayNumber')
    return jdata_df

jdata_train_df_201704 = getUserSkuActionDayNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserSkuActionDayNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserSkuActionDayNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserSkuActionDayNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserSkuActionDayNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserSkuActionDayNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserSkuActionDayNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的操作总数，逐个月进行处理
def getUserActionNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserActionNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_actionNumber')
        jdata_df = getUserActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_actionNumber')
        jdata_df = getUserActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_actionNumber')
        jdata_df = getUserActionNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_actionNumber')
    return jdata_df

jdata_train_df_201704 = getUserActionNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserActionNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserActionNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserActionNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserActionNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserActionNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserActionNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的浏览个数，逐个月进行处理
def getUserSkuBrowseNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 1)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 1)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'sku_id'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserSkuBrowseNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserSkuBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_skuBrowseNumber')
        jdata_df = getUserSkuBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_skuBrowseNumber')
        jdata_df = getUserSkuBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_skuBrowseNumber')
        jdata_df = getUserSkuBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_skuBrowseNumber')
    return jdata_df

jdata_train_df_201704 = getUserSkuBrowseNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserSkuBrowseNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserSkuBrowseNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserSkuBrowseNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserSkuBrowseNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserSkuBrowseNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserSkuBrowseNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的浏览天数，逐个月进行处理
def getUserSkuBrowseDayNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 1)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 1)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'a_date'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_date'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_date':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserSkuBrowseDayNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserSkuBrowseDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_skuBrowseDayNumber')
        jdata_df = getUserSkuBrowseDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_skuBrowseDayNumber')
        jdata_df = getUserSkuBrowseDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_skuBrowseDayNumber')
        jdata_df = getUserSkuBrowseDayNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_skuBrowseDayNumber')
    return jdata_df

jdata_train_df_201704 = getUserSkuBrowseDayNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserSkuBrowseDayNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserSkuBrowseDayNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserSkuBrowseDayNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserSkuBrowseDayNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserSkuBrowseDayNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserSkuBrowseDayNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的浏览总数，逐个月进行处理
def getUserBrowseNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 1)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 1)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserBrowseNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_browseNumber')
        jdata_df = getUserBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_browseNumber')
        jdata_df = getUserBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_browseNumber')
        jdata_df = getUserBrowseNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_browseNumber')
    return jdata_df

jdata_train_df_201704 = getUserBrowseNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserBrowseNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserBrowseNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserBrowseNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserBrowseNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserBrowseNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserBrowseNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的关注个数，逐个月进行处理
def getUserSkuFocusNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 2)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 2)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'sku_id'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserSkuFocusNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserSkuFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_skuFocusNumber')
        jdata_df = getUserSkuFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_skuFocusNumber')
        jdata_df = getUserSkuFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_skuFocusNumber')
        jdata_df = getUserSkuFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_skuFocusNumber')
    return jdata_df

jdata_train_df_201704 = getUserSkuFocusNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserSkuFocusNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserSkuFocusNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserSkuFocusNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserSkuFocusNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserSkuFocusNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserSkuFocusNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内对某个类目商品的关注总数，逐个月进行处理
def getUserFocusNumberMonthly(jdata_df, jdata_user_action_data, start_time, end_time, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 2)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 2)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date >= start_time) & (jdata_user_action_data.a_date < end_time) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserFocusNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 101, 'user_cate101_last' + str(month) + 'Month_focusNumber')
        jdata_df = getUserFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, 30, 'user_cate30_last' + str(month) + 'Month_focusNumber')
        jdata_df = getUserFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -1, 'user_targetCate_last' + str(month) + 'Month_focusNumber')
        jdata_df = getUserFocusNumberMonthly(jdata_df, jdata_user_action_data, featureMonthBegin, featureMonthEnd, -2, 'user_relatedCate_last' + str(month) + 'Month_focusNumber')
    return jdata_df

jdata_train_df_201704 = getUserFocusNumber(jdata_train_df_201704, jdata_user_action_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserFocusNumber(jdata_train_df_201703, jdata_user_action_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserFocusNumber(jdata_train_df_201702, jdata_user_action_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserFocusNumber(jdata_train_df_201701, jdata_user_action_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserFocusNumber(jdata_train_df_201612, jdata_user_action_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserFocusNumber(jdata_train_df_201611, jdata_user_action_data, FeatureMonthList_201611)
jdata_test_df = getUserFocusNumber(jdata_test_df, jdata_user_action_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户在滑窗区间内的评价次数，逐个月进行处理
def getUserCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, start_time, end_time, score_level, newColName):
    jdata_train_df_dealMonth = jdata_df
    if score_level == 0:
        jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.comment_create_tm >= start_time) & (jdata_user_comment_score_data.comment_create_tm < end_time)]
    else:
        jdata_user_comment_score_data_future = jdata_user_comment_score_data[(jdata_user_comment_score_data.comment_create_tm >= start_time) & (jdata_user_comment_score_data.comment_create_tm < end_time) & (jdata_user_comment_score_data.score_level == score_level)]
    jdata_user_comment_score_data_future_pivot_table = pd.pivot_table(jdata_user_comment_score_data_future, index=['user_id'], values=['score_level'], aggfunc=len)
    jdata_user_comment_score_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_comment_score_data_future_pivot_table.rename(columns={'score_level':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_comment_score_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    return jdata_train_df_dealMonth

def getUserCommentNumber(jdata_df, jdata_user_action_data, FeatureMonthList):
    for featureMonthBegin, featureMonthEnd, month in FeatureMonthList:
        jdata_df = getUserCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, featureMonthBegin, featureMonthEnd, 1, 'user_score1_last' + str(month) + 'Month_commentNumber')
        jdata_df = getUserCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, featureMonthBegin, featureMonthEnd, 2, 'user_score2_last' + str(month) + 'Month_commentNumber')
        jdata_df = getUserCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, featureMonthBegin, featureMonthEnd, 3, 'user_score3_last' + str(month) + 'Month_commentNumber')
        jdata_df = getUserCommentNumberMonthly(jdata_df, jdata_user_comment_score_data, featureMonthBegin, featureMonthEnd, 0, 'user_score0_last' + str(month) + 'Month_commentNumber')
    return jdata_df

jdata_train_df_201704 = getUserCommentNumber(jdata_train_df_201704, jdata_user_comment_score_data, FeatureMonthList_201704)
jdata_train_df_201703 = getUserCommentNumber(jdata_train_df_201703, jdata_user_comment_score_data, FeatureMonthList_201703)
jdata_train_df_201702 = getUserCommentNumber(jdata_train_df_201702, jdata_user_comment_score_data, FeatureMonthList_201702)
jdata_train_df_201701 = getUserCommentNumber(jdata_train_df_201701, jdata_user_comment_score_data, FeatureMonthList_201701)
jdata_train_df_201612 = getUserCommentNumber(jdata_train_df_201612, jdata_user_comment_score_data, FeatureMonthList_201612)
jdata_train_df_201611 = getUserCommentNumber(jdata_train_df_201611, jdata_user_comment_score_data, FeatureMonthList_201611)
jdata_test_df = getUserCommentNumber(jdata_test_df, jdata_user_comment_score_data, FeatureMonthList_201705)


In [ ]:
# 统计某用户距离上次购买商品的时间，逐个月进行处理
def getUserLastBuyDeltaTimeMonthly(jdata_df, jdata_user_order_data, year, month, cate, newColName):
    time_boundary = datetime.datetime(year, month, 1)
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date']][(jdata_user_order_data.o_date < time_boundary) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date']][(jdata_user_order_data.o_date < time_boundary) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date']][(jdata_user_order_data.o_date < time_boundary) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_future = jdata_user_order_data_future.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_user_order_data_future = jdata_user_order_data_future.drop_duplicates(['user_id'])
    jdata_user_order_data_future.rename(columns={'o_date':newColName}, inplace=True)
    jdata_df = pd.merge(jdata_df, jdata_user_order_data_future, on=['user_id'], how='left')
    jdata_df[newColName] = jdata_df[newColName].map(lambda x: 99999999 if pd.isnull(x) else (time_boundary - x).days)
    return jdata_df

def getUserLastBuyDeltaTime(jdata_df, jdata_user_order_data, year, month):
    jdata_df = getUserLastBuyDeltaTimeMonthly(jdata_df, jdata_user_order_data, year, month, -1, 'user_targetCate_lastBuyDeltaTime')
    jdata_df = getUserLastBuyDeltaTimeMonthly(jdata_df, jdata_user_order_data, year, month, -2, 'user_relatedCate_lastBuyDeltaTime')
    jdata_df = getUserLastBuyDeltaTimeMonthly(jdata_df, jdata_user_order_data, year, month, 101, 'user_cate101_lastBuyDeltaTime')
    jdata_df = getUserLastBuyDeltaTimeMonthly(jdata_df, jdata_user_order_data, year, month, 30, 'user_cate30_lastBuyDeltaTime')
    return jdata_df

jdata_train_df_201611 = getUserLastBuyDeltaTime(jdata_train_df_201611, jdata_user_order_data, 2016, 11)
jdata_train_df_201612 = getUserLastBuyDeltaTime(jdata_train_df_201612, jdata_user_order_data, 2016, 12)
jdata_train_df_201701 = getUserLastBuyDeltaTime(jdata_train_df_201701, jdata_user_order_data, 2017, 1)
jdata_train_df_201702 = getUserLastBuyDeltaTime(jdata_train_df_201702, jdata_user_order_data, 2017, 2)
jdata_train_df_201703 = getUserLastBuyDeltaTime(jdata_train_df_201703, jdata_user_order_data, 2017, 3)
jdata_train_df_201704 = getUserLastBuyDeltaTime(jdata_train_df_201704, jdata_user_order_data, 2017, 4)
jdata_test_df = getUserLastBuyDeltaTime(jdata_test_df, jdata_user_order_data, 2017, 5)



In [ ]:
# 统计某用户上次购买商品的数量，逐个月进行处理
def getUserLastBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, cate, newColName):
    time_boundary = datetime.datetime(year, month, 1)
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][(jdata_user_order_data.o_date < time_boundary) & ((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))]
        else:
            jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][(jdata_user_order_data.o_date < time_boundary) & ((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))]
    else:
        jdata_user_order_data_future = jdata_user_order_data[['user_id', 'o_date', 'o_sku_num']][(jdata_user_order_data.o_date < time_boundary) & (jdata_user_order_data.cate == cate)]
    jdata_user_order_data_future = jdata_user_order_data_future.sort_index(by=['user_id', 'o_date'], ascending=False)
    jdata_user_order_data_future = jdata_user_order_data_future.drop_duplicates(['user_id'])
    jdata_user_order_data_future.rename(columns={'o_sku_num':newColName}, inplace=True)
    jdata_df = pd.merge(jdata_df, jdata_user_order_data_future[['user_id', newColName]], on=['user_id'], how='left')
    jdata_df[newColName] = jdata_df[newColName].fillna(0)
    return jdata_df

def getUserLastBuyNumber(jdata_df, jdata_user_order_data, year, month):
    jdata_df = getUserLastBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, -1, 'user_targetCate_lastBuyNumber')
    jdata_df = getUserLastBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, -2, 'user_relatedCate_lastBuyNumber')
    jdata_df = getUserLastBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, 101, 'user_cate101_lastBuyNumber')
    jdata_df = getUserLastBuyNumberMonthly(jdata_df, jdata_user_order_data, year, month, 30, 'user_cate30_lastBuyNumber')
    return jdata_df

jdata_train_df_201611 = getUserLastBuyNumber(jdata_train_df_201611, jdata_user_order_data, 2016, 11)
jdata_train_df_201612 = getUserLastBuyNumber(jdata_train_df_201612, jdata_user_order_data, 2016, 12)
jdata_train_df_201701 = getUserLastBuyNumber(jdata_train_df_201701, jdata_user_order_data, 2017, 1)
jdata_train_df_201702 = getUserLastBuyNumber(jdata_train_df_201702, jdata_user_order_data, 2017, 2)
jdata_train_df_201703 = getUserLastBuyNumber(jdata_train_df_201703, jdata_user_order_data, 2017, 3)
jdata_train_df_201704 = getUserLastBuyNumber(jdata_train_df_201704, jdata_user_order_data, 2017, 4)
jdata_test_df = getUserLastBuyNumber(jdata_test_df, jdata_user_order_data, 2017, 5)



In [ ]:
# 统计用户上次购买时间间隔/数量的特征
def getUserLastDeltatimeNumber(jdata_df):
    jdata_df_copy = jdata_df.copy()
    jdata_df_copy1 = jdata_df_copy['user_targetCate_lastBuyNumber'].map(lambda x: 1 if x==0 else x)
    jdata_df['user_targetCate_lastDeltatimeNumber'] = jdata_df['user_targetCate_lastBuyDeltaTime'] / jdata_df_copy1
    jdata_df_copy2 = jdata_df_copy['user_relatedCate_lastBuyNumber'].map(lambda x: 1 if x==0 else x)
    jdata_df['user_relatedCate_lastDeltatimeNumber'] = jdata_df['user_relatedCate_lastBuyDeltaTime'] / jdata_df_copy2
    jdata_df_copy3 = jdata_df_copy['user_cate101_lastBuyNumber'].map(lambda x: 1 if x==0 else x)
    jdata_df['user_cate101_lastDeltatimeNumber'] = jdata_df['user_cate101_lastBuyDeltaTime'] / jdata_df_copy3
    jdata_df_copy4 = jdata_df_copy['user_cate30_lastBuyNumber'].map(lambda x: 1 if x==0 else x)
    jdata_df['user_cate30_lastDeltatimeNumber'] = jdata_df['user_cate30_lastBuyDeltaTime'] / jdata_df_copy4
    return jdata_df

jdata_train_df_201611 = getUserLastDeltatimeNumber(jdata_train_df_201611)
jdata_train_df_201612 = getUserLastDeltatimeNumber(jdata_train_df_201612)
jdata_train_df_201701 = getUserLastDeltatimeNumber(jdata_train_df_201701)
jdata_train_df_201702 = getUserLastDeltatimeNumber(jdata_train_df_201702)
jdata_train_df_201703 = getUserLastDeltatimeNumber(jdata_train_df_201703)
jdata_train_df_201704 = getUserLastDeltatimeNumber(jdata_train_df_201704)
jdata_test_df = getUserLastDeltatimeNumber(jdata_test_df)


In [ ]:
# 统计某用户平均每个月购买某个类目商品的次数，逐个月进行处理
def getUserMonthlyBuyNumberMonthly(jdata_df, jdata_user_order_data, cate, newColName, time_boundary, length):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)) & (jdata_user_order_data.o_date < time_boundary)]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)) & (jdata_user_order_data.o_date < time_boundary)]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyBuyNumber(jdata_df, jdata_user_order_data, time_boundary, length):
    jdata_df = getUserMonthlyBuyNumberMonthly(jdata_df, jdata_user_order_data, 101, 'user_101Cate_monthlyBuyNumber', time_boundary, length)
    jdata_df = getUserMonthlyBuyNumberMonthly(jdata_df, jdata_user_order_data, 30, 'user_30Cate_monthlyBuyNumber', time_boundary, length)
    jdata_df = getUserMonthlyBuyNumberMonthly(jdata_df, jdata_user_order_data, -1, 'user_targetCate_monthlyBuyNumber', time_boundary, length)
    jdata_df = getUserMonthlyBuyNumberMonthly(jdata_df, jdata_user_order_data, -2, 'user_relatedCate_monthlyBuyNumber', time_boundary, length)
    return jdata_df

jdata_train_df_201704 = getUserMonthlyBuyNumber(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyBuyNumber(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyBuyNumber(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyBuyNumber(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyBuyNumber(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyBuyNumber(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyBuyNumber(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 5, 1), 12)



In [ ]:
# 统计某用户平均每个月购买某个类目商品的下单次数，逐个月进行处理
def getUserMonthlyOrderNumberMonthly(jdata_df, jdata_user_order_data, cate, newColName, time_boundary, length):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)) & (jdata_user_order_data.o_date < time_boundary)]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)) & (jdata_user_order_data.o_date < time_boundary)]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_buy_future.drop_duplicates(['user_id', 'o_id'], inplace=True)
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_id'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyOrderNumber(jdata_df, jdata_user_order_data, time_boundary, length):
    jdata_df = getUserMonthlyOrderNumberMonthly(jdata_df, jdata_user_order_data, 101, 'user_101Cate_monthlyOrderNumber', time_boundary, length)
    jdata_df = getUserMonthlyOrderNumberMonthly(jdata_df, jdata_user_order_data, 30, 'user_30Cate_monthlyOrderNumber', time_boundary, length)
    jdata_df = getUserMonthlyOrderNumberMonthly(jdata_df, jdata_user_order_data, -1, 'user_targetCate_monthlyOrderNumber', time_boundary, length)
    jdata_df = getUserMonthlyOrderNumberMonthly(jdata_df, jdata_user_order_data, -2, 'user_relatedCate_monthlyOrderNumber', time_boundary, length)
    return jdata_df

jdata_train_df_201704 = getUserMonthlyOrderNumber(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyOrderNumber(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyOrderNumber(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyOrderNumber(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyOrderNumber(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyOrderNumber(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyOrderNumber(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月购买某个类目商品的件数，逐个月进行处理
def getUserMonthlyBuyCountMonthly(jdata_df, jdata_user_order_data, cate, newColName, time_boundary, length):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)) & (jdata_user_order_data.o_date < time_boundary)]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)) & (jdata_user_order_data.o_date < time_boundary)]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_sku_num'], aggfunc=sum)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_sku_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyBuyCount(jdata_df, jdata_user_order_data, time_boundary, length):
    jdata_df = getUserMonthlyBuyCountMonthly(jdata_df, jdata_user_order_data, 101, 'user_101Cate_monthlyBuyCount', time_boundary, length)
    jdata_df = getUserMonthlyBuyCountMonthly(jdata_df, jdata_user_order_data, 30, 'user_30Cate_monthlyBuyCount', time_boundary, length)
    jdata_df = getUserMonthlyBuyCountMonthly(jdata_df, jdata_user_order_data, -1, 'user_targetCate_monthlyBuyCount', time_boundary, length)
    jdata_df = getUserMonthlyBuyCountMonthly(jdata_df, jdata_user_order_data, -2, 'user_relatedCate_monthlyBuyCount', time_boundary, length)
    return jdata_df

jdata_train_df_201704 = getUserMonthlyBuyCount(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyBuyCount(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyBuyCount(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyBuyCount(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyBuyCount(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyBuyCount(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyBuyCount(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月购买某个类目商品的天数，逐个月进行处理
def getUserMonthlyBuyDayMonthly(jdata_df, jdata_user_order_data, cate, newColName, time_boundary, length):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30)) & (jdata_user_order_data.o_date < time_boundary)]
        else:
            jdata_user_order_data_buy_future = jdata_user_order_data[((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30)) & (jdata_user_order_data.o_date < time_boundary)]
    else:
        jdata_user_order_data_buy_future = jdata_user_order_data[(jdata_user_order_data.cate == cate) & (jdata_user_order_data.o_date < time_boundary)]
    jdata_user_order_data_buy_future.drop_duplicates(['user_id', 'o_date'], inplace=True)
    jdata_user_order_data_buy_future_pivot_table = pd.pivot_table(jdata_user_order_data_buy_future, index=['user_id'], values=['o_date'], aggfunc=len)
    jdata_user_order_data_buy_future_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_buy_future_pivot_table.rename(columns={'o_date':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_order_data_buy_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyBuyDay(jdata_df, jdata_user_order_data, time_boundary, length):
    jdata_df = getUserMonthlyBuyDayMonthly(jdata_df, jdata_user_order_data, 101, 'user_101Cate_monthlyBuyDay', time_boundary, length)
    jdata_df = getUserMonthlyBuyDayMonthly(jdata_df, jdata_user_order_data, 30, 'user_30Cate_monthlyBuyDay', time_boundary, length)
    jdata_df = getUserMonthlyBuyDayMonthly(jdata_df, jdata_user_order_data, -1, 'user_targetCate_monthlyBuyDay', time_boundary, length)
    jdata_df = getUserMonthlyBuyDayMonthly(jdata_df, jdata_user_order_data, -2, 'user_relatedCate_monthlyBuyDay', time_boundary, length)
    return jdata_df

jdata_train_df_201704 = getUserMonthlyBuyDay(jdata_train_df_201704, jdata_user_order_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyBuyDay(jdata_train_df_201703, jdata_user_order_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyBuyDay(jdata_train_df_201702, jdata_user_order_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyBuyDay(jdata_train_df_201701, jdata_user_order_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyBuyDay(jdata_train_df_201612, jdata_user_order_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyBuyDay(jdata_train_df_201611, jdata_user_order_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyBuyDay(jdata_test_df, jdata_user_order_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月对某个类目商品的关注总数，逐个月进行处理
def getUserMonthlyFocusNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 2)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 2)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 2)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyFocusNumber(jdata_df, jdata_user_action_data, time_boundary, length):
    jdata_df = getUserMonthlyFocusNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 101, 'user_cate101_monthlyFocusNumber')
    jdata_df = getUserMonthlyFocusNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 30, 'user_cate30_monthlyFocusNumber')
    jdata_df = getUserMonthlyFocusNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -1, 'user_targetCate_monthlyFocusNumber')
    jdata_df = getUserMonthlyFocusNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -2, 'user_relatedCate_monthlyFocusNumber')
    return jdata_df

jdata_train_df_201704 = getUserMonthlyFocusNumber(jdata_train_df_201704, jdata_user_action_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyFocusNumber(jdata_train_df_201703, jdata_user_action_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyFocusNumber(jdata_train_df_201702, jdata_user_action_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyFocusNumber(jdata_train_df_201701, jdata_user_action_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyFocusNumber(jdata_train_df_201612, jdata_user_action_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyFocusNumber(jdata_train_df_201611, jdata_user_action_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyFocusNumber(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月对某个类目商品的浏览总数，逐个月进行处理
def getUserMonthlyBrowseNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30)) & (jdata_user_action_data.a_type == 1)]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30)) & (jdata_user_action_data.a_type == 1)]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.cate == cate) & (jdata_user_action_data.a_type == 1)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyBrowseNumber(jdata_df, jdata_user_action_data, time_boundary, length):
    jdata_df = getUserMonthlyBrowseNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 101, 'user_cate101_monthlyBrowseNumber')
    jdata_df = getUserMonthlyBrowseNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 30, 'user_cate30_monthlyBrowseNumber')
    jdata_df = getUserMonthlyBrowseNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -1, 'user_targetCate_monthlyBrowseNumber')
    jdata_df = getUserMonthlyBrowseNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -2, 'user_relatedCate_monthlyBrowseNumber')
    return jdata_df

jdata_train_df_201704 = getUserMonthlyBrowseNumber(jdata_train_df_201704, jdata_user_action_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyBrowseNumber(jdata_train_df_201703, jdata_user_action_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyBrowseNumber(jdata_train_df_201702, jdata_user_action_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyBrowseNumber(jdata_train_df_201701, jdata_user_action_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyBrowseNumber(jdata_train_df_201612, jdata_user_action_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyBrowseNumber(jdata_train_df_201611, jdata_user_action_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyBrowseNumber(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月对某个类目商品的操作总数，逐个月进行处理
def getUserMonthlyActionNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_num'], aggfunc=sum)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_num':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyActionNumber(jdata_df, jdata_user_action_data, time_boundary, length):
    jdata_df = getUserMonthlyActionNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 101, 'user_cate101_monthlyActionNumber')
    jdata_df = getUserMonthlyActionNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 30, 'user_cate30_monthlyActionNumber')
    jdata_df = getUserMonthlyActionNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -1, 'user_targetCate_monthlyActionNumber')
    jdata_df = getUserMonthlyActionNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -2, 'user_relatedCate_monthlyActionNumber')
    return jdata_df

jdata_train_df_201704 = getUserMonthlyActionNumber(jdata_train_df_201704, jdata_user_action_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyActionNumber(jdata_train_df_201703, jdata_user_action_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyActionNumber(jdata_train_df_201702, jdata_user_action_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyActionNumber(jdata_train_df_201701, jdata_user_action_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyActionNumber(jdata_train_df_201612, jdata_user_action_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyActionNumber(jdata_train_df_201611, jdata_user_action_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyActionNumber(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月对某个类目商品的操作天数，逐个月进行处理
def getUserMonthlyActionDayNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'a_date'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['a_date'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'a_date':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyActionDayNumber(jdata_df, jdata_user_action_data, time_boundary, length):
    jdata_df = getUserMonthlyActionDayNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 101, 'user_cate101_monthlyActionDayNumber')
    jdata_df = getUserMonthlyActionDayNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 30, 'user_cate30_monthlyActionDayNumber')
    jdata_df = getUserMonthlyActionDayNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -1, 'user_targetCate_monthlyActionDayNumber')
    jdata_df = getUserMonthlyActionDayNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -2, 'user_relatedCate_monthlyActionDayNumber')
    return jdata_df

jdata_train_df_201704 = getUserMonthlyActionDayNumber(jdata_train_df_201704, jdata_user_action_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyActionDayNumber(jdata_train_df_201703, jdata_user_action_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyActionDayNumber(jdata_train_df_201702, jdata_user_action_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyActionDayNumber(jdata_train_df_201701, jdata_user_action_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyActionDayNumber(jdata_train_df_201612, jdata_user_action_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyActionDayNumber(jdata_train_df_201611, jdata_user_action_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyActionDayNumber(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计某用户平均每个月对某个类目商品的操作商品数，逐个月进行处理
def getUserMonthlyActionSkuNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, cate, newColName):
    jdata_train_df_dealMonth = jdata_df
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate == 101) | (jdata_user_action_data.cate == 30))]
        else:
            jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & ((jdata_user_action_data.cate != 101) & (jdata_user_action_data.cate != 30))]
    else:
        jdata_user_action_data_future = jdata_user_action_data[(jdata_user_action_data.a_date < time_boundary) & (jdata_user_action_data.cate == cate)]
    jdata_user_action_data_future.drop_duplicates(['user_id', 'sku_id'], inplace=True)
    jdata_user_action_data_future_pivot_table = pd.pivot_table(jdata_user_action_data_future, index=['user_id'], values=['sku_id'], aggfunc=len)
    jdata_user_action_data_future_pivot_table.reset_index(inplace=True)
    jdata_user_action_data_future_pivot_table.rename(columns={'sku_id':newColName}, inplace=True)
    jdata_train_df_dealMonth = pd.merge(jdata_train_df_dealMonth, jdata_user_action_data_future_pivot_table, on=['user_id'], how='left')
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName].fillna(0)
    jdata_train_df_dealMonth[newColName] = jdata_train_df_dealMonth[newColName] / length
    return jdata_train_df_dealMonth

def getUserMonthlyActionSkuNumber(jdata_df, jdata_user_action_data, time_boundary, length):
    jdata_df = getUserMonthlyActionSkuNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 101, 'user_cate101_monthlyActionSkuCount')
    jdata_df = getUserMonthlyActionSkuNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, 30, 'user_cate30_monthlyActionSkuCount')
    jdata_df = getUserMonthlyActionSkuNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -1, 'user_targetCate_monthlyActionSkuCount')
    jdata_df = getUserMonthlyActionSkuNumberMonthly(jdata_df, jdata_user_action_data, time_boundary, length, -2, 'user_relatedCate_monthlyActionSkuCount')
    return jdata_df

jdata_train_df_201704 = getUserMonthlyActionSkuNumber(jdata_train_df_201704, jdata_user_action_data, datetime.datetime(2017, 4, 1), 11)
jdata_train_df_201703 = getUserMonthlyActionSkuNumber(jdata_train_df_201703, jdata_user_action_data, datetime.datetime(2017, 3, 1), 10)
jdata_train_df_201702 = getUserMonthlyActionSkuNumber(jdata_train_df_201702, jdata_user_action_data, datetime.datetime(2017, 2, 1), 9)
jdata_train_df_201701 = getUserMonthlyActionSkuNumber(jdata_train_df_201701, jdata_user_action_data, datetime.datetime(2017, 1, 1), 8)
jdata_train_df_201612 = getUserMonthlyActionSkuNumber(jdata_train_df_201612, jdata_user_action_data, datetime.datetime(2016, 12, 1), 7)
jdata_train_df_201611 = getUserMonthlyActionSkuNumber(jdata_train_df_201611, jdata_user_action_data, datetime.datetime(2016, 11, 1), 6)
jdata_test_df = getUserMonthlyActionSkuNumber(jdata_test_df, jdata_user_action_data, datetime.datetime(2017, 5, 1), 12)


In [ ]:
# 统计用户平均购买时间间隔特征
def getUserBuyDeltaTimeFeatureMonthly(jdata_df, jdata_user_order_data, cate):
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            jdata_user_order_data_copy = jdata_user_order_data[((jdata_user_order_data.cate == 101) | (jdata_user_order_data.cate == 30))].copy()
        else:
            jdata_user_order_data_copy = jdata_user_order_data[((jdata_user_order_data.cate != 101) & (jdata_user_order_data.cate != 30))].copy()
    else:
        jdata_user_order_data_copy = jdata_user_order_data[(jdata_user_order_data.cate == cate)].copy()
    jdata_user_order_data_copy.drop_duplicates(['user_id', 'o_date'], inplace=True)
    jdata_user_order_data_copy = jdata_user_order_data_copy.sort_index(by=['user_id', 'o_date'], ascending=True)
    jdata_user_order_data_copy['last_user_id'] = jdata_user_order_data_copy['user_id'].shift(1)
    jdata_user_order_data_copy['last_user_id'] = jdata_user_order_data_copy['user_id'] == jdata_user_order_data_copy['last_user_id']
    jdata_user_order_data_copy['last_o_date'] = jdata_user_order_data_copy['o_date'].shift(1)
    jdata_user_order_data_copy_ave = jdata_user_order_data_copy[jdata_user_order_data_copy.last_user_id == True]
    jdata_user_order_data_copy_ave['user_deltaTimeAve'] = jdata_user_order_data_copy_ave['o_date'] - jdata_user_order_data_copy_ave['last_o_date']
    jdata_user_order_data_copy_ave['user_deltaTimeAve'] = jdata_user_order_data_copy_ave['user_deltaTimeAve'].map(lambda x: x.days)
    jdata_user_order_data_copy_ave_pivot_table = pd.pivot_table(jdata_user_order_data_copy_ave, index=['user_id'], values=['user_deltaTimeAve'], aggfunc={min, max, mean})
    jdata_user_order_data_copy_ave_pivot_table.reset_index(inplace=True)
    jdata_user_order_data_copy_ave_pivot_table.columns = ['user_id', 'user_cate'+ str(cate) + '_deltaTime_max', 'user_cate' + str(cate) + '_deltaTime_mean', 'user_cate' + str(cate) + '_deltaTime_min']
    jdata_df = pd.merge(jdata_df, jdata_user_order_data_copy_ave_pivot_table, on=['user_id'], how='left')
    return jdata_df

def getUserBuyDeltaTimeFeature(jdata_df, jdata_user_order_data):
    jdata_df = getUserBuyDeltaTimeFeatureMonthly(jdata_df, jdata_user_order_data, -1)
    jdata_df = getUserBuyDeltaTimeFeatureMonthly(jdata_df, jdata_user_order_data, -2)
    jdata_df = getUserBuyDeltaTimeFeatureMonthly(jdata_df, jdata_user_order_data, 101)
    jdata_df = getUserBuyDeltaTimeFeatureMonthly(jdata_df, jdata_user_order_data, 30)
    return jdata_df

jdata_train_df_201704 = getUserBuyDeltaTimeFeature(jdata_train_df_201704, jdata_user_order_data)
jdata_train_df_201703 = getUserBuyDeltaTimeFeature(jdata_train_df_201703, jdata_user_order_data)
jdata_train_df_201702 = getUserBuyDeltaTimeFeature(jdata_train_df_201702, jdata_user_order_data)
jdata_train_df_201701 = getUserBuyDeltaTimeFeature(jdata_train_df_201701, jdata_user_order_data)
jdata_train_df_201612 = getUserBuyDeltaTimeFeature(jdata_train_df_201612, jdata_user_order_data)
jdata_train_df_201611 = getUserBuyDeltaTimeFeature(jdata_train_df_201611, jdata_user_order_data)
jdata_test_df = getUserBuyDeltaTimeFeature(jdata_test_df, jdata_user_order_data)


In [ ]:
# 统计用户上次购买商品时间间隔－平均购买时间间隔特征
def getUserBuyDeltaTimeLastAve(jdata_df):
    jdata_df['user_cate30_last-ave'] = jdata_df['user_cate30_lastBuyDeltaTime'] - jdata_df['user_cate30_deltaTime_mean'] 
    jdata_df['user_cate101_last-ave'] = jdata_df['user_cate101_lastBuyDeltaTime'] - jdata_df['user_cate101_deltaTime_mean'] 
    jdata_df['user_cate-1_last-ave'] = jdata_df['user_targetCate_lastBuyDeltaTime'] - jdata_df['user_cate-1_deltaTime_mean'] 
    jdata_df['user_cate-2_last-ave'] = jdata_df['user_relatedCate_lastBuyDeltaTime'] - jdata_df['user_cate-2_deltaTime_mean'] 
    return jdata_df

jdata_train_df_201704 = getUserBuyDeltaTimeLastAve(jdata_train_df_201704)
jdata_train_df_201703 = getUserBuyDeltaTimeLastAve(jdata_train_df_201703)
jdata_train_df_201702 = getUserBuyDeltaTimeLastAve(jdata_train_df_201702)
jdata_train_df_201701 = getUserBuyDeltaTimeLastAve(jdata_train_df_201701)
jdata_train_df_201612 = getUserBuyDeltaTimeLastAve(jdata_train_df_201612)
jdata_train_df_201611 = getUserBuyDeltaTimeLastAve(jdata_train_df_201611)
jdata_test_df = getUserBuyDeltaTimeLastAve(jdata_test_df)



In [ ]:
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    print('origion_rate_mean : ', origion_rate_mean)
    print('origion_rate_var : ', origion_rate_var)
    print('alpha : ', alpha)
    print('beta : ', beta)
    return alpha, beta

# 添加用户一个月前购买数量占三个月内购买数量的比例特征,进行贝叶斯平滑
def getUserBuyNumberLastOneInThreeRatioMonthly(jdata_df, cate):
    if ((cate == -1) | (cate == -2)):
        if cate == -1:
            ratioName = 'user_targetCate_buyNumber_lastOneInThreeRatio'
            threeMonthNumberName = 'user_targetCate_last3Month_buyNumber'
            oneMonthNumberName = 'user_targetCate_last1Month_buyNumber'
        else:
            ratioName = 'user_relatedCate_buyNumber_lastOneInThreeRatio'
            threeMonthNumberName = 'user_relatedCate_last3Month_buyNumber'
            oneMonthNumberName = 'user_relatedCate_last1Month_buyNumber'
    else:
        ratioName = 'user_cate' + str(cate) + '_buyNumber_lastOneInThreeRatio'
        threeMonthNumberName = 'user_cate' + str(cate) + '_last3Month_buyNumber'
        oneMonthNumberName = 'user_cate' + str(cate) + '_last1Month_buyNumber'
    jdata_df_copy = jdata_df.copy()
    jdata_df_copy[ratioName] = -1
    jdata_df_copy[ratioName][jdata_df_copy[threeMonthNumberName] != 0] = jdata_df_copy[oneMonthNumberName][jdata_df_copy[threeMonthNumberName] != 0] / jdata_df_copy[threeMonthNumberName][jdata_df_copy[threeMonthNumberName] != 0] 
    alpha, beta = getBayesSmoothParam(jdata_df_copy[ratioName][jdata_df_copy[ratioName] != -1])
    jdata_df_copy[ratioName] = (jdata_df_copy[oneMonthNumberName] + alpha) / (jdata_df_copy[threeMonthNumberName] + alpha + beta)
    jdata_df = pd.merge(jdata_df, jdata_df_copy[['user_id', ratioName]], on=['user_id'], how='left')
    return jdata_df

def getUserBuyNumberLastOneInThreeRatio(jdata_df):
    jdata_df = getUserBuyNumberLastOneInThreeRatioMonthly(jdata_df, 101)
    jdata_df = getUserBuyNumberLastOneInThreeRatioMonthly(jdata_df, 30)
    jdata_df = getUserBuyNumberLastOneInThreeRatioMonthly(jdata_df, -1)
    jdata_df = getUserBuyNumberLastOneInThreeRatioMonthly(jdata_df, -2)
    return jdata_df

jdata_train_df_201704 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201704)
jdata_train_df_201703 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201703)
jdata_train_df_201702 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201702)
jdata_train_df_201701 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201701)
jdata_train_df_201612 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201612)
jdata_train_df_201611 = getUserBuyNumberLastOneInThreeRatio(jdata_train_df_201611)
jdata_test_df = getUserBuyNumberLastOneInThreeRatio(jdata_test_df)


In [ ]:
print(len(jdata_train_df_201704))
print(len(jdata_train_df_201704.columns.values))
print(jdata_train_df_201704.columns.values)


In [ ]:
jdata_train_df_baseline = pd.concat([jdata_train_df_201611, jdata_train_df_201612, jdata_train_df_201701, jdata_train_df_201702, jdata_train_df_201703, jdata_train_df_201704])
print(len(jdata_train_df_baseline))


In [ ]:
# 导出训练集和测试集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)
    
exportResult(jdata_train_df_baseline, 'chusai_jdata_train_df_baseline')
exportResult(jdata_test_df, 'chusai_jdata_test_df_baseline')
